references: 
- https://openai.github.io/openai-agents-python/tools/#agents-as-tools

In [1]:
import os
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from openai import AsyncOpenAI, AsyncAzureOpenAI
from agents import (

    Agent,
    Runner,
    OpenAIChatCompletionsModel,
    ModelProvider,
    Model,
    RunConfig,
    set_default_openai_client,
    set_default_openai_api,
    set_tracing_disabled,
    set_tracing_export_api_key,
    trace,


    function_tool,
)
load_dotenv()

True

In [2]:
client = AsyncOpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["GITHUB_TOKEN"],
)


# client = AsyncAzureOpenAI(
#     base_url=os.getenv("AZURE_OPENAI_ENDPOINT"),
#     # azure_deployment="gpt-4o",
#     api_version="2025-03-01-preview",
#     api_key=os.getenv("AZURE_OPENAI_API_KEY")
# )

set_default_openai_client(client)
set_default_openai_api(
    ['chat_completions']
)
set_tracing_export_api_key(os.environ["OPENAI_TRACING_KEY"])
set_tracing_disabled(False)

class GitHubModelProvider(ModelProvider):
    def get_model(self, model_name) -> Model:
        return OpenAIChatCompletionsModel(model=model_name, openai_client=client)



GITHUB_MODEL_PROVIDER = GitHubModelProvider()



# tr = trace(workflow_name="msdiagentic_hello_tracking")

## define agents

In [15]:
model_name = "gpt-4o"


spanish_agent = Agent(
    name="Spanish agent",
    instructions="You translate the user's message to Spanish",
    model=model_name,

)

french_agent = Agent(
    name="French agent",
    instructions="You translate the user's message to French",
    model=model_name
)

orchestrator_agent = Agent(
    name="orchestrator_agent",
    model=model_name,
    instructions=(
        "You are a translation agent. You use the tools given to you to translate."
        "If asked for multiple translations, you call the relevant tools."
        "You only speak English. Do not translate the user's message yourself."
        "You can use the tools to translate the user's message."
    ),
    tools=[
        spanish_agent.as_tool(
            tool_name="translate_to_spanish",
            tool_description="Translate the user's message to Spanish",
        ),
        french_agent.as_tool(
            tool_name="translate_to_french",
            tool_description="Translate the user's message to French",
        ),
    ],
)

## execute agents

In [16]:
orchestrator_agent.model

'gpt-4o'

In [17]:
try:
    with trace(workflow_name="translation_agent_as_tools", group_id="mdsi") as tr:
        result = await Runner.run(
            orchestrator_agent,
            "Say 'Hello, how are you?' in Spanish.",
            run_config=RunConfig(model=model_name, model_provider=GITHUB_MODEL_PROVIDER),

        )
except Exception as e:
    print(f"Error: {e}")


print(result.final_output)


Error: Error code: 429 - {'error': {'code': 'RateLimitReached', 'message': 'Rate limit of 150 per 86400s exceeded for UserByModelByDay. Please wait 38187 seconds before retrying.', 'details': 'Rate limit of 150 per 86400s exceeded for UserByModelByDay. Please wait 38187 seconds before retrying.'}}
I’m sorry, but I’m having trouble with the translation tool. Please try again later.


In [18]:
tracing_url = f"https://platform.openai.com/traces/trace?trace_id={tr.trace_id}"

print(f"Tracing URL: {tracing_url}")


Tracing URL: https://platform.openai.com/traces/trace?trace_id=trace_fd64a901640244d19a1896af5b74feea


## Visualise the Agent

In [22]:
from agents.extensions.visualization import draw_graph

try:
    draw_graph(orchestrator_agent, filename="viz/06_agent_as_tool.gv")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH
